In [ ]:
from neutrino_level.steps.general_modules.read_data import load_dfs

In [ ]:
from glob import glob
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
from copy import copy

In [ ]:
import helper
traj = helper.build_trajectory()

In [ ]:
mc_df, data_df = load_dfs(
    traj, ignore_tables=helper.to_ignore)

In [ ]:
from stochasticity_observable_ import get_1d_rlogl
from stochasticity_observable_ import spline, Likelihood_1D

sys.path.append('/Users/brianclark/Documents/work/IceCube/ehe/ehe_deps/energy_loss_pdfs')
from likelihood import Likelihood_1D

get_1d_rlogl(
    mc_df,
    Likelihood_1D(spline),
    table_name='EHEMuMillipede_SplineMPEseed_vecd',
    key_name='stochasticity',
    min_bins=8,
    n_bins_to_combine=4)

In [ ]:
stoch_thresh = 8.374043045935741
rlogl_mask = mc_df['stochasticity'] >= stoch_thresh

In [ ]:
def track_quality_cut(speed=None, npe=None):
    '''
    Input: np.ndarrays of values to base the cut on
    Returns: mask, where "True" indicates an event *PASSES* the cut
    '''
    # pass charge cut
    mask1 = npe > 10**4.65
    
    # does *NOT* pass the track speed cut below 0.26 (cascade like)
    low_speed = speed <= 0.26
    low_mask = np.logical_and(low_speed, npe < 10**5.25)
    
    # does *NOT* pass the track speed cut between 0.26 and 0.28
    intermediate_speed = np.logical_and(speed > 0.26,
                                    speed < 0.28)
    intermediate_charge_thresh = (
        np.power(10, 5.25 - (0.6/0.02) * (speed - 0.26)))
    intermediate_mask = np.logical_and(
        intermediate_speed,
        npe < intermediate_charge_thresh)
      
    # total mask (mask1 and NOT low mask and NOT intermediate mask)
    total_mask = np.logical_and(
        mask1, ~intermediate_mask
    )
    total_mask = np.logical_and(
       total_mask, ~low_mask 
    )
    return total_mask


def muon_bundle_cut_stoch_opt(
        mc_df,
        reco_zen_key='EHEOpheliaParticleSRT_ImpLF.zenith',
        npe_key='EHEPortiaEventSummary.atwdNPEbtw',
        inplace=False,
        floor=4.55,
        ceil=6.05,
        denom=0.9,
        inflection_point=0.1,
        pwr=2.5):
    # Apply base cut
    mask = mc_df[npe_key] >= 10**floor
    
    # Add zenith dependent npe cut
    above_horizon = np.cos(mc_df[reco_zen_key]) >= inflection_point
    npe_thresh = np.power(10, floor + (ceil-floor) *
                          np.sqrt(1 - np.power((1-np.cos(mc_df[reco_zen_key]))/denom, pwr)))
    npe_mask = np.logical_and(
        above_horizon,
        mc_df[npe_key] < npe_thresh)
    mask = np.logical_and(mask, ~npe_mask)
    return mask

In [ ]:
params_low = {
    'floor': 4.7,
    'inflection_point': 0.1,
    'ceil': 6.25,
    'denom': 0.9,
    'pwr': 1.5
}

params_high = {
    'floor': 4.65,
    'inflection_point': 0.1,
    'ceil': 5.7,
    'denom': 0.9,
    'pwr': 4
}

In [ ]:
bundle_mask_low = np.logical_and(
    muon_bundle_cut_stoch_opt(
        mc_df,
        reco_zen_key='EHE_SplineMPE.zenith',
        npe_key='Homogenized_QTot.value',
        **params_low
    ),
    ~rlogl_mask
)

bundle_mask_high = np.logical_and(
    muon_bundle_cut_stoch_opt(
        mc_df,
        reco_zen_key='EHE_SplineMPE.zenith',
        npe_key='Homogenized_QTot.value',
        **params_high
    ),
    rlogl_mask
)

bundle_mask = np.logical_or(
    bundle_mask_low,
    bundle_mask_high
)

track_quality_mask = track_quality_cut(
    mc_df['EHELineFit.speed'],
    mc_df['Homogenized_QTot.value']
)

selection_mask = np.logical_and(
    bundle_mask,
    track_quality_mask
)

In [ ]:
params_old = {
    'floor': 4.6,
    'inflection_point': 0.06,
    'ceil': 6.45,
    'denom': 0.94,
    'pwr': 2.
}

## Test if just doing the low stoch cut will approximately reproduce old effective areas
bundle_mask_check = muon_bundle_cut_stoch_opt(
    mc_df,
    reco_zen_key='EHE_SplineMPE.zenith',
    npe_key='Homogenized_QTot.value',
    **params_old
)

selection_mask_check = np.logical_and(
    bundle_mask_check,
    track_quality_mask
)

In [ ]:
final_mc_df = mc_df.loc[selection_mask]

In [ ]:
from neutrino_level.steps.general_modules.juliet_weighting import calc_juliet_effective_area
from neutrino_level.steps.general_modules.juliet_weighting import get_juliet_weightdict_and_propmatrix
import tables
e_bins = np.logspace(5, 12, 141)

In [ ]:
def correct_events_per_file(juliet_species, juliet_energy_level):
    n_he = 150
    n_vhe = 20
    nu_scaling = 4
    
    evts_per_file = -1
    
    if juliet_species in ['nue', 'numu', 'nutau']:
        if juliet_energy_level == 'vhe':
            evts_per_file = n_vhe * nu_scaling
        else:
            evts_per_file = n_he * nu_scaling
    else:
        if juliet_energy_level == 'vhe':
            if juliet_species == 'tau':
                evts_per_file = 100
            else:
                evts_per_file = n_vhe
        else:
            evts_per_file = n_he
    
    return evts_per_file



run_ids = mc_df.index.get_level_values(0)

eff_area_dict = {}
eff_area_dict_check = {}

for comp in traj.parameters.components:
    print(comp._name)
    species, energy_level = comp._name.split('_')
    for file_i, n_files_i in zip(comp.file_list, comp.n_files_loaded):
        with tables.open_file(file_i) as open_file:
            weight_dict, prop_matrix, evts_per_file = \
                get_juliet_weightdict_and_propmatrix(open_file)
            evts_per_file = correct_events_per_file(species, energy_level)
            primary_energy = open_file.get_node(
                '/MCPrimary').col('energy')
            log_energy_max = np.log10(np.max(primary_energy))
            
            if comp.type.endswith('nue'):
                pdg_id = 12
            elif comp.type.endswith('numu'):
                pdg_id = 14
            elif comp.type.endswith('nutau'):
                pdg_id = 16
            elif comp.type.endswith('mu'):
                pdg_id = 13
            elif comp.type.endswith('tau'):
                pdg_id = 15

            ds_id_base = pdg_id * 100000
            if log_energy_max > 9.0:
                ds_id_base += 10000
                
            run_id_mask = np.logical_and(
                run_ids >= ds_id_base,
                run_ids < ds_id_base + 10000
            )

            mask = selection_mask[run_id_mask]
            mask_ = selection_mask_check[run_id_mask]
            
            for prop_matrix_flavor, flavor in zip(
                    prop_matrix, ['nue', 'numu', 'nutau']):

                eff_area = calc_juliet_effective_area(
                    energies=primary_energy,
                    weight_dict=weight_dict,
                    n_gen=evts_per_file*n_files_i,
                    energy_bins=e_bins,
                    prop_matrix=prop_matrix_flavor,
                    selection_mask=mask
                )
                eff_area_dict[comp._name + f'_{flavor}'] = eff_area.sum(axis=0)
                
                eff_area_check = calc_juliet_effective_area(
                    energies=primary_energy,
                    weight_dict=weight_dict,
                    n_gen=evts_per_file*n_files_i,
                    energy_bins=e_bins,
                    prop_matrix=prop_matrix_flavor,
                    selection_mask=mask_
                )
                eff_area_dict_check[comp._name + f'_{flavor}'] = eff_area_check.sum(axis=0)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
areas_from_nue = [eff_area_dict[key] for key in eff_area_dict.keys() if key.endswith('nue')]
areas_from_numu = [eff_area_dict[key] for key in eff_area_dict.keys() if key.endswith('numu')]
areas_from_nutau = [eff_area_dict[key] for key in eff_area_dict.keys() if key.endswith('nutau')]

areas_from_nue_check = [eff_area_dict_check[key] for key in eff_area_dict_check.keys() if key.endswith('nue')]
areas_from_numu_check = [eff_area_dict_check[key] for key in eff_area_dict_check.keys() if key.endswith('numu')]
areas_from_nutau_check = [eff_area_dict_check[key] for key in eff_area_dict_check.keys() if key.endswith('nutau')]

In [ ]:
bin_center = (e_bins[1:] + e_bins[:-1]) / 2
bin_width = np.diff(e_bins)

fig, ax = plt.subplots()
for i, (areas, label_i) in enumerate(zip([areas_from_nue, areas_from_numu, areas_from_nutau],
                                        ['NuE', 'NuMu', 'NuTau'])):
    ax.plot(bin_center, np.sum(areas, axis=0), drawstyle='steps-mid',
            color=f'C{i}')
    ax.errorbar(bin_center, np.sum(areas, axis=0),
                xerr=bin_width/2.,
                label=f'{label_i}',
                fmt='.', markersize=0,
                color=f'C{i}')
for i, (areas, label_i) in enumerate(zip([areas_from_nue_check, areas_from_numu_check, areas_from_nutau_check],
                                        ['NuE', 'NuMu', 'NuTau'])):
    ax.plot(bin_center, np.sum(areas, axis=0), drawstyle='steps-mid',
            color=f'C{i}', ls='--')
    ax.errorbar(bin_center, np.sum(areas, axis=0),
                xerr=bin_width/2.,
                label=f'{label_i}',
                fmt='.', markersize=0,
                color=f'C{i}', ls='--')
ax.legend()
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(5e5, 1e11)
ax.set_ylim(1e0, 5e4)
ax.set_xlabel(r'$E_{\nu}$ / GeV')
ax.set_ylabel(r'$A_{\mathrm{eff}} \,\, / \,\, \mathrm{m}^2$')


In [ ]:
total_aeff = np.sum(areas_from_nue, axis=0) + np.sum(areas_from_numu, axis=0) + np.sum(areas_from_nutau, axis=0)

# average the effective areas together to get total sensitivity to the neutrino flux
total_aeff = total_aeff/3. # make flavor average

# smooth out the effective area a bit
bin_center = bin_center[5:-4] # drop 10 bins
total_aeff = np.convolve(total_aeff, np.ones(10)/10, mode='valid') # average
total_aeff = total_aeff * CM2_TO_M2 # convert to m2
total_aeff_sr = total_aeff * 4 * np.pi # convert to m2 str (mult by 4pi)

np.savez('ehenextgen_total_aeff.npz',
         bin_center=bin_center,
         avg_aeff_m2sr=total_aeff_sr
         )

from old_analysis_results import diff_limit_9yr, diff_limit_7yr, LIVETIME_9YR
from old_analysis_results import CM2_TO_M2
LIVETIME_12YR = 12 * 365 * 24 * 3600
from ehefluxes import fluxes

flux_spl = fluxes.EHEFlux('ahlers_gzk')
flux_spl_2010 = fluxes.EHEFlux('cosmogenic_ahlers2010_1E18')

In [ ]:
def simple_e2_differential_limit(energies, total_aeff_sr, livetime):
        # input 
        # energies (in real units, eV)
        # flavor averaged effective area (in m2)
        # livetime (in seconds)
        
        # returns
        # E^2 * F(E) (GeV/cm2/s/sr)
        n_90 = 2.44
        return n_90 * energies / (livetime * np.log(10) * total_aeff_sr)

fig, ax = plt.subplots()

ax.plot(bin_center,
        simple_e2_differential_limit(bin_center,
                                  total_aeff_sr,
                                  LIVETIME_9YR),
        label='New EHE 8.15yrs IC86 equiv, sensitivity')
ax.plot(bin_center,
        simple_e2_differential_limit(bin_center,
                                  total_aeff_sr,
                                  LIVETIME_12YR),
        label='New EHE 12yrs, sensitivity')
ax.plot(diff_limit_9yr[:, 0], diff_limit_9yr[:, 1], label='EHE 9yrs', color='k')
ax.plot(diff_limit_7yr[:, 0], diff_limit_7yr[:, 1], label='EHE 7yrs', color='k', ls='--')

energies = np.logspace(5, 12, 141)

ax.plot(energies, energies**2 * flux_spl(energies, 'sum'), label='Ahlers 2012')
ax.plot(energies, energies**2 * flux_spl_2010(energies, 'sum'), label='Ahlers 2010 1 EeV')

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('E / GeV')
ax.legend(bbox_to_anchor=(1.,0.85,0.,0))
ax.set_ylim(1e-10, 1e-6)
ax.set_xlim(1e6, 1e11)
ax.set_ylabel(r'E$^2 \cdot \Phi$ / (GeV cm$^{-2}$ sr$^{-1}$ s$^{-1}$)')

In [ ]:
fig, ax = plt.subplots()
ax.plot(bin_center,
        simple_e2_differential_limit(bin_center,
                                  total_aeff_sr,
                                  LIVETIME_12YR),
        label='EHE UL 12yrs, sensitivity (this work)')
ax.plot(diff_limit_9yr[:, 0], diff_limit_9yr[:, 1], label='EHE UL 9yrs', color='k')
ax.plot(diff_limit_7yr[:, 0], diff_limit_7yr[:, 1], label='EHE UL 7yrs', color='k', ls='--')

energies = np.logspace(5, 12, 141)

ax.plot(energies, energies**2 * flux_spl_2010(energies, 'sum'), label='Ahlers 2010 1 EeV', color='grey')
ax.plot(energies, energies**2 * flux_spl(energies, 'sum'), label='Ahlers 2012', color='grey', ls='--')

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('E / GeV')
ax.legend(bbox_to_anchor=(1.,0.85,0.,0))
ax.set_ylim(1e-10, 5e-7)
ax.set_xlim(5e6, 1e11)
ax.set_ylabel(r'E$^2 \cdot \Phi$ / (GeV cm$^{-2}$ sr$^{-1}$ s$^{-1}$)')
ax.text(6e6, 3e-7, 'IceCube Work-in-progress', color='red')